

This Module is responsible for:


*   **Hyperparameter tuning** (*hyperparameters of CNN-GRU including Number of filters, Kernel size, Pool size, Number of GRU units*) using GWO-WOA and comparing with other metaheuristic optimizers 
*   **Classifying the Sentiments of documents using the CNN-GRU**



# Configuration

In [ ]:
! pip install sklearn-deap
! pip install np
! pip install sklearn
! pip install sklearn_nature_inspired_algorithms==0.4.3

In [ ]:
pip install Niapy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn_nature_inspired_algorithms.model_selection.nature_inspired_search_cv import NatureInspiredSearchCV
from sklearn_nature_inspired_algorithms.helpers import score_by_generation_lineplot

from evolutionary_search import EvolutionaryAlgorithmSearchCV

In [ ]:

import string
from __future__ import print_function

import keras
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense, Dropout, Embedding, LSTM, GRU, Bidirectional, Activation, Conv1D, MaxPooling1D, GlobalMaxPooling1D, BatchNormalization
from keras.optimizers import SGD
from keras.layers import SpatialDropout1D, Flatten
from keras.callbacks import Callback
from keras.callbacks import ReduceLROnPlateau
from keras import backend
from keras.initializers import Constant
from keras.regularizers import l2
from sklearn.metrics import precision_recall_fscore_support as score

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from matplotlib import pyplot as plt
from matplotlib.pyplot import plot



import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
w2vMovieTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.txt'
w2vMovieM = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/w2vMovie-MR-99-07-25.h5'

lstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.txt'
lstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/lstm-99-07-25.h5'

gruModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.txt'
gruModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/gru-99-07-25.h5'

bilstmModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.txt'
bilstmModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/bilstm-99-07-25.h5'

conv1ModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.txt'
conv1ModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/conv1-99-07-25.h5'

CLModelTxt = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.txt'
CLModelH = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Cl-99-07-25.h5'

#This file includes tweets and topic number
tweetsTopics = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Tweets_Topics_990725.csv'

#mr8Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/JokerMostSimilar-99-07-26.csv'
mr8Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_8_MostSimilar.csv'
mr20Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_20_MostSimilar.csv'
mr40Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_40_MostSimilar.csv'
mr80Similar = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Topics_80_MostSimilar.csv'






#allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/M-A-S140-Processed-98-11-09-03.csv'
#allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/movie-pang-processed-98-12-08.csv'

allTrainingData = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/Processed-Merged-Dataset-99-08-28.csv'
pangLabelledDataset = '/MYDRIVE/My Drive/Colab Notebooks/Proposal/Data/MovieReview-Preprocessed-Pang-Ver2-Labelled.csv'

# Hyperparameters

In [ ]:
#Making Word2vec
embeddingDim = 500
maxLen = embeddingDim
testSize=0.2
windowSize=5
minCount=1 
iterCount=10
sG = 0 #skip gram wil be used


#Making Model
validationSplit = 0.2
#maxFeatures = numWords
maxLen = 500
embeddingSize = 100
isTrainable = False
batchSize = 32



epochsNum = 100

unitsNum = 20


# Convolution
#kernelSize = 3
kernelSize = 4

filtersNum = 500


poolSize = 2
stridesNum = 1
hidden_dims = 100

# LSTM
lstm_output_size = 70

# Log in to Google Drive

In [ ]:
#No need in local
from google.colab import drive
drive.mount('/MYDRIVE', force_remount=True)

# Loading Data to make Word2Vec Model

In [ ]:
dfMR8 = pd.DataFrame()
dfMR8 = pd.read_csv(mr80Similar, encoding = 'utf-8')
print(dfMR8.head(3))
dfMR81 = dfMR8
print(len(dfMR81))

print(dfMR81.columns)

dfMR81.columns = [ 'i', 'docid','Probability']

print(dfMR81.columns)

print(dfMR81.head(3))

print(len(dfMR81))

In [ ]:
len(dfMR81['docid'].unique())

# Loading All Training Labelled Data

In [ ]:
allTrainingData

In [ ]:
dfALLTraining = pd.DataFrame()
dfALLTraining = pd.read_csv(allTrainingData, encoding = 'utf-8', header=None)
print(dfALLTraining.head(3))
print(len(dfALLTraining))

# Finding movie docs from training data

In [ ]:
#This data is similar to topic 11
dfTrainingMovies = pd.DataFrame()

dfTrainingMovies = dfALLTraining.loc[dfALLTraining[0].isin(dfMR81['docid'])]
print(len(dfTrainingMovies))
print(dfTrainingMovies.head(3))

# Making Word2Vec Model

In [ ]:
#embeddingDim = 100
#Remove repeated rows
dfMoviesNoRep = dfTrainingMovies
dfMoviesNoRep.columns = ['i','TweetId', 'Tweet', 'Sentiment']
print(dfMoviesNoRep.head(3))  
dfMoviesNoRep.sort_values(["TweetId"], inplace = True) 
print(dfMoviesNoRep.head(3))  
# dropping ALL duplicte values 
dfMoviesNoRep.drop_duplicates(subset ="TweetId", 
                     keep = 'first', inplace = True) 
print(len(dfMoviesNoRep))

In [ ]:
print(len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0]))
print(len(dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1]))

In [ ]:
# Making a balance dataset with equal number of positive and negative tweets
df1 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 1].sample(14089)
df2 = dfMoviesNoRep[dfMoviesNoRep['Sentiment'] == 0].sample(14089)
df = df1.append(df2)
print(len(df[df['Sentiment'] == 1]))
print(df.head())
print(len(df))

In [ ]:
dfPang = pd.DataFrame()
dfPang = pd.read_csv(pangLabelledDataset, encoding = 'utf-8', header=None)
print(dfPang.head(3))
print(len(dfPang))
dfPang.insert(0,'i',0)
dfPang.columns = ['i' , 'TweetId', 'Tweet', 'Sentiment']
df = df.append(dfPang)
print(len(df))
print(len(df[df['Sentiment'] == 1]))
print(len(df[df['Sentiment'] == 0]))

In [ ]:
from sklearn.model_selection import train_test_split
x_train_m, x_test_m, y_train_m, y_test_m = train_test_split( df['Tweet'], df['Sentiment'], test_size=0.2, random_state=10)

In [ ]:
print(len(x_test_m))
print(len(x_train_m))

In [ ]:
print(len(y_test_m[y_test_m == 0]))
print(len(y_test_m[y_test_m == 1]))

In [ ]:
print(len(y_train_m[y_train_m == 0]))
print(len(y_train_m[y_train_m == 1]))

In [ ]:
reviewMovieLines = list()
for line in x_train_m:
  reviewMovieLines.append( str(line).split() )

for line in x_test_m:
  reviewMovieLines.append( str(line).split() )

print(len(reviewMovieLines))

In [ ]:
import gensim


w2vMovie = gensim.models.Word2Vec( sentences=reviewMovieLines,  window=windowSize, vector_size = embeddingDim, workers=4, min_count=minCount, epochs=iterCount, sg = sG )

w2vGensimModel = gensim.models.Word2Vec()
#vocab size

#words = list( w2vMovie.wv.vocab )

#print('Vocabulary size: %d' % len(words))

In [ ]:
w2vMovie.wv.save_word2vec_format( w2vMovieTxt, binary = False)
w2vMovie.save(w2vMovieM)

In [ ]:
w2vMovie.wv.most_similar('film')

In [ ]:
w2vMovie.wv.most_similar('movi')

# Using Word2Vec to input network

In [ ]:
import os
embeddingsIndex = {}
f = open(os.path.join('', w2vMovieTxt ), encoding = "utf-8")

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:])
  embeddingsIndex[word] = coefs
f.close()

In [ ]:
tokenizerObj = Tokenizer()
tokenizerObj.fit_on_texts(reviewMovieLines)
sequences = tokenizerObj.texts_to_sequences(reviewMovieLines)

print(type(sequences))

reviewPad = pad_sequences( sequences, maxlen=maxLen)

wordIndex = tokenizerObj.word_index
print('Found %s unique tokens.' % len(wordIndex))

#reviewPad = pad_sequences( sequences, maxlen=maxLen)
#sentiment = df ['Sentiment'].values
print(type(reviewPad))
print(len(reviewPad))

x_train_pad = reviewPad[ : len(x_train_m)]
x_test_pad = reviewPad[ len(x_train_m) : ]
y_train = y_train_m
y_test = y_test_m

print(type(x_train_pad))

print('Shape of x_train_pad tensor:' , len(x_train_pad))
print('Shape of y_train tensor:' , len(y_train))


print('Shape of x_test_pad tensor:' , len(x_test_pad))
print('Shape of y_test tensor:' , len( y_test ))


In [ ]:
x = y_test[y_test == 0]
print(len(x))

In [ ]:
x = y_test[y_test == 1]
print(len(x))

In [ ]:
x = y_train[y_train == 0]
print(len(x))
x = y_train[y_train == 1]
print(len(x))

In [ ]:
numWords = len(wordIndex) + 1
print(numWords)
embeddingMatrix = np.zeros( (numWords, embeddingDim) )
#print(embeddingMatrix.shape)
for word, i in wordIndex.items():
  if i > numWords:
    continue
  embeddingVector = embeddingsIndex.get(word)
  if embeddingVector is not None:
    # words not found in embedding index will be all-zeros.
    embeddingMatrix[i] = embeddingVector

In [ ]:
print(embeddingMatrix.shape)

In [ ]:
max_features = numWords

# HPO Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn import datasets
import scipy.stats as stats
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping


In [ ]:
from sklearn_nature_inspired_algorithms.model_selection.nature_inspired_search_cv import NatureInspiredSearchCV
from sklearn_nature_inspired_algorithms.helpers import score_by_generation_lineplot

#  CNN + GRU

In [ ]:
#def CNNGRU(neurons1=20, neurons2=10,batch_size=500,epochs=20,activation='relu',patience=3 ,loss='binary_crossentropy'):
def CNNGRU(numFilters = 20, kernelSize = 5, poolSize = 2, gruUnits = 20):
    print('Build model...')
    modelCGRU = Sequential(name='CNNGRU')
    modelCGRU.add(Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable))
    modelCGRU.add(Conv1D(filters= numFilters, kernel_size= kernelSize, strides= stridesNum, padding='same', activation='relu'))
    modelCGRU.add(Dropout(0.4))
    modelCGRU.add(MaxPooling1D(pool_size = poolSize))
    modelCGRU.add(Dropout(0.6))
    modelCGRU.add(GRU( gruUnits , dropout=0.6 ))
    modelCGRU.add(Dropout(0.6))
    modelCGRU.add(Dense(1, activation='sigmoid'))
    modelCGRU.compile(loss='binary_crossentropy',
              optimizer= 'adam',
              metrics=['accuracy'])
    print('Train...')

    early_stopping = EarlyStopping(monitor="loss", patience = 10)# early stop patience

    historyCGRU = modelCGRU.fit(x_train_pad, y_train,
          batch_size=32,
          epochs=100,
          callbacks = [early_stopping],
          validation_data=(x_test_pad, y_test), verbose = 0 )#, callbacks=[rlrp])    
    
    
    return modelCGRU

In [ ]:
rf_params = {
    'numFilters': [32, 64],
    'kernelSize':[3, 4, 5, 6, 7],
    'poolSize':[2, 3, 4],
    'gruUnits':[10, 15, 20, 25]
}

# GA

In [ ]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space

clf = KerasClassifier(build_fn=CNNGRU, verbose=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="accuracy",
                                   cv=3,
                                   verbose=1,
                                   population_size=20,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(x_train_pad, y_train)
print(ga1.best_params_)
print("Accuracy:"+ str(ga1.best_score_))


# PSO

In [ ]:
from NiaPy.algorithms.basic import ParticleSwarmOptimization
#clf = RandomForestClassifier(random_state=42)
clf = KerasClassifier(build_fn=CNNGRU, verbose=0)

algorithm = ParticleSwarmOptimization() # when custom algorithm is provided random_state is ignored

nia_search = NatureInspiredSearchCV(
  clf,
    rf_params,
    cv=3,
    verbose=1,
    algorithm=algorithm,
    population_size=20,
    runs=5,
    n_jobs=-1,
    scoring='accuracy')
nia_search.fit(x_train_pad, y_train)
print(nia_search.best_params_)
print("accuracy:"+ str(nia_search.best_score_))

# FA

In [ ]:
%%time

clf = KerasClassifier(build_fn=CNNGRU, verbose=0)

nia_search = NatureInspiredSearchCV(
    clf,
    rf_params,
    cv=3,
    verbose=1,
    algorithm='fa',
    population_size=20,
    n_jobs=-1   
    ,scoring='accuracy')
nia_search.fit(x_train_pad, y_train)
print(nia_search.best_params_)
print("accuracy:"+ str(nia_search.best_score_))

# GWO

In [ ]:
%%time

clf = KerasClassifier(build_fn=CNNGRU, verbose=0)

nia_search = NatureInspiredSearchCV(
    clf,
    rf_params,
    cv=3,
    verbose=1,
    algorithm='gwo',
    population_size=20,
    runs=5,
    n_jobs=-1,
   scoring='accuracy')
nia_search.fit(x_train_pad, y_train)
print(nia_search.best_params_)
print("accuracy:"+ str(nia_search.best_score_))

# GWO-WOA

In [ ]:
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.metrics import mean_squared_error , mean_absolute_error, accuracy_score
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from tqdm import tqdm
from scipy import stats
from statsmodels.tsa.statespace.varmax import VARMAX

import statsmodels.api as sm

import IPython
import IPython.display


import warnings 
warnings.filterwarnings('ignore')

In [ ]:
#Making DNN
import tensorflow as tf
def CNNGRU2(numFilters = 20, kernelSize = 5, poolSize = 2, gruUnits = 20):
    CNN_GRU_Model = tf.keras.Sequential([
    tf.keras.layers.Embedding(numWords, embeddingDim , 
                      embeddings_initializer = Constant( embeddingMatrix), 
                      input_length=maxLen,
                      trainable = isTrainable)
    ,
    tf.keras.layers.Conv1D(filters= numFilters, kernel_size= kernelSize, strides= 1, padding='same', activation='relu')
    ,
    tf.keras.layers.Dropout(0.4)
    ,
    tf.keras.layers.MaxPooling1D(pool_size = poolSize)
    ,
    tf.keras.layers.Dropout(0.4)
    ,
    tf.keras.layers.GRU( gruUnits , dropout=0.6 )
    ,
    tf.keras.layers.Dropout(0.6)
    ,
    tf.keras.layers.Dense(1, activation='sigmoid')

])

    #LSTM_MLP_Model.compile(optimizer = "adam", loss=loss, metrics=[tf.metrics.MeanAbsoluteError()])
    CNN_GRU_Model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                      patience=10)
    
    hist = CNN_GRU_Model.fit(x_train_pad, y_train, epochs=100, batch_size=32, validation_data=(x_test_pad, y_test), verbose = 0) 
    
    return hist,CNN_GRU_Model

In [ ]:
rf_params = {
    'numFilters': [32, 64],
    'kernelSize':[3, 4, 5, 6, 7],
    'poolSize':[2, 3, 4],
    'gruUnits':[10, 15, 20, 25]
}

In [ ]:
# sphere function
def shpere_fun(x):
    z=np.sum(x**2)
    return z

In [ ]:
def get_integer_value(x,L):
  div_val=np.linspace(0, 1, L)
  idx=np.argwhere(x>=div_val)
  return idx[-1][0]

In [ ]:
def cost_fun(x):
  
  # get par base x value
  select_par=dict()
  counter=0
  for key in rf_params:
    val_key=rf_params.get(key)# get value each params
    L=len(val_key)
    idx_slecect=get_integer_value(x[counter],L) # get integer idx by x value
    select_par[key]=val_key[idx_slecect] # add select param to select_par
    counter+=1
  print(select_par)
  hist,model=CNNGRU2(**select_par)
  accuracy=hist.history['accuracy'][-1]
  
  return accuracy

In [ ]:
def get_para(x):
  
  # get par base x value
  select_par=dict()
  counter=0
  for key in rf_params:
    val_key=rf_params.get(key)# get value each params
    L=len(val_key)
    idx_slecect=get_integer_value(x[counter],L) # get integer idx by x value
    select_par[key]=val_key[idx_slecect] # add select param to select_par
    counter+=1
  return select_par

In [ ]:
select_par=get_para([32, 3 ,4, 50])
select_par
#{'activation': 'relu', 'neurons1': 20, 'neurons2': 25}

## GWO-pop

In [ ]:
def GWO_POP(objf, lb, ub, dim, SearchAgents_no, it,Positions,
            Alpha_pos,Alpha_score,Beta_pos,Beta_score,Delta_pos,Delta_score):
  l=it
  for i in range(0, SearchAgents_no):
    # Return back the search agents that go beyond the boundaries of the search space
    for j in range(dim):
        Positions[i, j] = numpy.clip(Positions[i, j], lb[j], ub[j])

    # Calculate objective function for each search agent
    fitness = objf(Positions[i, :])
    # Update Alpha, Beta, and Delta
    if fitness < Alpha_score:
        Delta_score = Beta_score  # Update delte
        Delta_pos = Beta_pos.copy()
        Beta_score = Alpha_score  # Update beta
        Beta_pos = Alpha_pos.copy()
        Alpha_score = fitness
        # Update alpha
        Alpha_pos = Positions[i, :].copy()

    if fitness > Alpha_score and fitness < Beta_score:
        Delta_score = Beta_score  # Update delte
        Delta_pos = Beta_pos.copy()
        Beta_score = fitness  # Update beta
        Beta_pos = Positions[i, :].copy()

    if fitness > Alpha_score and fitness > Beta_score and fitness < Delta_score:
        Delta_score = fitness  # Update delta
        Delta_pos = Positions[i, :].copy()

    a = 2 - (l+1) * ((2) / Max_iter)
    # a decreases linearly fron 2 to 0

    # Update the Position of search agents including omegas
    for i in range(0, SearchAgents_no):
      for j in range(0, dim):

          r1 = numpy.random.rand()  # r1 is a random number in [0,1]
          r2 = numpy.random.rand()  # r2 is a random number in [0,1]

          A1 = 2 * a * r1 - a
          # Equation (3.3)
          C1 = 2 * r2
          # Equation (3.4)

          D_alpha = abs(C1 * Alpha_pos[j] - Positions[i, j])
          # Equation (3.5)-part 1
          X1 = Alpha_pos[j] - A1 * D_alpha
          # Equation (3.6)-part 1

          r1 = numpy.random.rand()
          r2 = numpy.random.rand()

          A2 = 2 * a * r1 - a
          # Equation (3.3)
          C2 = 2 * r2
          # Equation (3.4)

          D_beta = abs(C2 * Beta_pos[j] - Positions[i, j])
          # Equation (3.5)-part 2
          X2 = Beta_pos[j] - A2 * D_beta
          # Equation (3.6)-part 2

          r1 = numpy.random.rand()
          r2 = numpy.random.rand()

          A3 = 2 * a * r1 - a
          # Equation (3.3)
          C3 = 2 * r2
          # Equation (3.4)

          D_delta = abs(C3 * Delta_pos[j] - Positions[i, j])
          # Equation (3.5)-part 3
          X3 = Delta_pos[j] - A3 * D_delta
          # Equation (3.5)-part 3

          Positions[i, j] = (X1 + X2 + X3) / 3  # Equation (3.7)




  return Positions,Alpha_pos,Alpha_score,Beta_pos,Beta_score,Delta_pos,Delta_score


## WOA-POP

In [ ]:
def WOA_POP(objf, lb, ub, dim, SearchAgents_no, t,Positions,Leader_pos,Leader_score):


  for i in range(0, SearchAgents_no):
      for j in range(dim):
          Positions[i, j] = np.clip(Positions[i, j], lb[j], ub[j])
          
    # Calculate objective function for each search agent
      fitness = objf(Positions[i, :])

        # Update Alpha, Beta, and Delta
      if fitness < Leader_score:
          Leader_score = fitness  # Update delte
          Leader_pos = Positions[i, :].copy()


  a = 2-(t+1)*((2)/Max_iter);           # linearly decreased from 2 to 0
  a2=-1+(t+1)*((-1)/Max_iter);
  
  
  
  for i in range(SearchAgents_no):
      r1 = np.random.rand()
      r2 = np.random.rand()
      A = 2 * a * r1 - a
      C = 2 * r2
      l = (a2-1)*np.random.rand()+1
      b=1
      p = np.random.rand()
      for j in range(dim):
        
      
          if ( p<0.5) :
              

            if(np.abs(A)>=1) :
                rand_leader_index = int(np.floor(SearchAgents_no*np.random.rand()+1)-1)
                x_rand = Positions[rand_leader_index,:]
                D_X_rand = np.abs(C * x_rand[j] - Positions[i][j])
                Positions[i,j] = (x_rand[j] - A *D_X_rand)
            elif(np.abs(A) < 1):
                D_Leader = np.abs(C * Leader_pos[j]- Positions[i][j] )
                Positions[i,j] = Leader_pos[j] - A * D_Leader
          elif(p>=0.5):
            distance2Leader = np.abs(Leader_pos[j]- Positions[i][j])
            Positions[i,j] = distance2Leader * np.exp(b * l) * np.cos(2 * np.pi * l) + Leader_pos[j]



  

  return Positions,Leader_pos,Leader_score

## Hybrid Serial GWO-WOA

In [ ]:
#setting parameter
Max_iter=1
lb=0
ub=1
dim=len(rf_params)
SearchAgents_no=4
objf=cost_fun
# initialize alpha, beta, and delta_pos
Alpha_pos = numpy.zeros(dim)
Alpha_score = float("inf")
Beta_pos = numpy.zeros(dim)
Beta_score = float("inf")
Delta_pos = numpy.zeros(dim)
Delta_score = float("inf")

if not isinstance(lb, list):
    lb = [lb] * dim
if not isinstance(ub, list):
    ub = [ub] * dim

# Initialize the positions of search agents
Positions = numpy.zeros((SearchAgents_no, dim))
for i in range(dim):
    Positions[:, i] = (numpy.random.uniform(0, 1, SearchAgents_no) * (ub[i] - lb[i]) + lb[i])

#------main loop
Convergence_curve = numpy.zeros(Max_iter)

for it in range(Max_iter):
  Positions,Alpha_pos,Alpha_score,Beta_pos,Beta_score,Delta_pos,Delta_score=GWO_POP(objf, lb, ub, dim, SearchAgents_no, it,Positions,
                              Alpha_pos,Alpha_score,Beta_pos,Beta_score,Delta_pos,Delta_score)
  
  Positions,Alpha_pos,Alpha_score=WOA_POP(objf, lb, ub, dim, SearchAgents_no, it,Positions,Alpha_pos,Alpha_score)
  Convergence_curve[it] = Alpha_score

  print(["At iteration " + str(it) + " the best fitness is " + str(Alpha_score)])

serial_GWO_WOA_parar=get_para(Alpha_pos)
print('\n_______________best parameter_____________________')
print(get_para(Alpha_pos))
print('accuracy: {0}'.format(Alpha_score))